# Chapter 2 — Building your first agents 

## 1. Quick Intro

In this chapter we’ll meet three **foundational building blocks** of the OpenAI Agents SDK and wire them together in a tiny demo (adapted from the official example).

## What are the primitives?

- **Agent**  
  A large language model **with a role/instructions and optional tools**.  
  Think “specialized workers” (e.g., *Writer*, *Researcher*, *Editor*).

- **Handoffs**  
  A structured way for one agent to **delegate a sub-task to another agent**.  
  This lets you compose multi-step workflows (e.g., Research → Write → Review).

- **Guardrails**  
  **Validation checks** around what goes **into** (and optionally out of) an agent.  
  Use them to enforce constraints (length, JSON schema, safety rules, etc.).

## What we’ll build (mini example)

We’ll implement a minimal **three-stage flow** that exercises all primitives:

1) A **Writer Agent** drafts a short answer.  
2) It **handoffs** to an **Editor Agent** for clarity/length polishing.  
3) Before finalizing, a **Guardrail** checks the output (e.g., “≤ 120 words”).  
4) Return the clean, validated result.

You’ll see how each primitive stays small and composable:
- Agents encapsulate **instructions + model**.
- Handoffs express **who should act next**.
- Guardrails keep the pipeline **safe and predictable**.



## 1. Setup

In [ ]:
import os
from dotenv import load_dotenv
from agents.extensions.models.litellm_model import LitellmModel

# Load env
load_dotenv()
api_key = os.getenv('API_KEY')

base_url = "https://api.openai.com/v1"  # We use openai's model here
chat_model = "gpt-4.1-nano-2025-04-14"   # We will be using cheaper model as im broke AF
emb_model = "text-embedding-3-small"

llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

In [13]:
from agents import set_tracing_disabled
set_tracing_disabled(True)

## 2. Define Your First Agent
An **Agent** needs at least:
- `name` — who this agent is (for logs/tracing & readability)
- `instructions` — how it should behave (role, tone, boundaries)

You can also attach a **model** per agent. Here we reuse the `llm` you configured earlier (e.g., a LiteLLM-backed provider). Each agent can share the same model **or** use a different one.

In [ ]:
from agents import Agent
agent = Agent(
    name="Math Tutor",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm, # uses the model configured in the previous step
)

#### Why this matters

- Clear `instructions` = consistent behavior.

- `model=llm` lets you override per agent (e.g., Math uses Claude, Editor uses GPT).

- Easy to mix providers across agents without changing the rest of your pipeline.

## 3. Add More Agents (with `handoff_description`)

Now let’s add a couple of **specialist agents** and describe **when others should hand off** to them.  
`handoff_description` is a short, routing-friendly blurb that helps the system decide **who** should take over a sub-task.



In [4]:
from agents import Agent

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

#### Why this helps

- Clear `handoff_description` → easier delegation (other agents know who to call).
 
- Each agent can share the same `llm` or use a different model tuned to its domain.
 
- Keep descriptions short and unambiguous (e.g., “math questions,” “historical questions”).

## 4. Define Handoffs (the “Project Manager”)

**Handoffs** let an agent delegate to others. Here we add a **Triage Agent** that decides *who should handle the user’s question* — like a project manager routing tasks.



In [14]:
# Define a router/triage agent that can hand off to specialists

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=llm,
)

### Run !

You can now run the triage → specialist flow.   
The triage agent will select the proper tutor and return the final answer.

In [15]:
result = await Runner.run(triage_agent, "What is the capital of France?")
print(result.final_output)

The capital of France is Paris. Paris is not only the largest city in France but also historically significant as a cultural, political, and economic center. It has been an important city throughout history, especially during the Middle Ages and the Renaissance, and is renowned for landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum.


As a script (.py)

```python
from agents import Runner
import asyncio

async def main():
    result = await Runner.run(triage_agent, "What is the capital of France?")
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

```

## 5. Add Guardrails (validate inputs before routing)

Guardrails let you **inspect/validate** a request **before** an agent acts.  
Here we add an **input guardrail** that checks: “Is this a homework question?” If not, we trigger a **tripwire** and stop the flow.

### What the code does
- **`HomeworkOutput` (Pydantic)**: structured result from a checker agent (`is_homework`, `reasoning`).
- **`guardrail_agent`**: a small classifier-style agent that answers in that schema.
- **`homework_guardrail`**: runs the checker agent, then returns `GuardrailFunctionOutput`.
  - `tripwire_triggered=True` ⇒ block the request.
- **`triage_agent`**: adds `input_guardrails=[InputGuardrail(...)]` so every incoming user query is screened **before** handoffs.


In [16]:
from agents import GuardrailFunctionOutput, InputGuardrail, Agent, Runner
from pydantic import BaseModel

class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
    model=llm,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

In [ ]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],

    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),
    ],
    model=llm,
)

#### What happens at runtime

- If the user asks, e.g., “What is the capital of France?” → `is_homework=True` → triage continues and delegates.

- If the user asks, “Does drinking tea good for the body?” → `is_homework=False` → guardrail trips and raisess error.

In [19]:
# result = await Runner.run(triage_agent, "does drinking tee good for the body?")
# print(result.final_output)

#### How to handle the error 

If you prefer a graceful message instead of an exception, catch it and reply:

In [20]:
from agents.exceptions import InputGuardrailTripwireTriggered

try:
    result = await Runner.run(triage_agent, "does drinking tea good for the body?")
    print(result.final_output)
except InputGuardrailTripwireTriggered as e:
    print("This assistant only handles homework questions. Please ask a math or history homework question. 🙏")


This assistant only handles homework questions. Please ask a math or history homework question. 🙏


Expecept output :"This assistant only handles homework questions. Please ask a math or history homework question. 🙏"

## 5. Complete Example (Agents · Handoffs · Guardrail)

Below is the **all-in-one script** you can run as a `.py` file.  
It wires up two specialist agents (History, Math), a **triage** agent that delegates via **handoffs**, and an **input guardrail** that only allows *homework* questions.

```python
from agents import Agent, Runner, set_tracing_disabled, GuardrailFunctionOutput, InputGuardrail
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv
from pydantic import BaseModel

# --- Config & Model ---
load_dotenv()
api_key = os.getenv("mistral_key")  # set in your .env (e.g., mistral_key=sk-xxxx)
base_url = "https://api.mistral.ai/v1"
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)  # optional: disable SDK tracing/export
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# --- Guardrail schema ---
class HomeworkOutput(BaseModel):
    is_homework: bool
    reasoning: str

# --- Specialist agents ---
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

# --- Guardrail checker agent ---
guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the user is asking about homework.",
    output_type=HomeworkOutput,
    model=llm,
)

async def homework_guardrail(ctx, agent, input_data):
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(HomeworkOutput)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.is_homework,
    )

# --- Triage (router) agent with handoffs + input guardrail ---
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    input_guardrails=[InputGuardrail(guardrail_function=homework_guardrail)],
    model=llm,
)

# --- Run (as a script) ---
async def main():
    result = await Runner.run(triage_agent, "does drinking tee good for the body?")
    print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())
